<a href="https://colab.research.google.com/github/gogowonji/080239/blob/master/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==1.15

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.7.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.7.0
     |████████████████████████████████| 412.3 MB 24 kB/s 
     |████████████████████████████████| 3.8 MB 38.5 MB/s 
     |████████████████████████████████| 503 kB 42.7 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=3d0d7bca25cc212d7856c92509e911237738505833d0040e84d944b4961fcc42
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [ ]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0000651
n_hidden = 256
n_hidden2 = 128
n_input = 28 * 28
n_noise = 128

In [ ]:
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_hidden2], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_hidden2]))
G_W3 = tf.Variable(tf.random_normal([n_hidden2, n_hidden], stddev=0.01))
G_b3 = tf.Variable(tf.zeros([n_hidden]))
G_W4 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b4 = tf.Variable(tf.zeros([n_input]))

D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, n_hidden2], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([n_hidden2]))
D_W3 = tf.Variable(tf.random_normal([n_hidden2, 1], stddev=0.01))
D_b3 = tf.Variable(tf.zeros([1]))

In [ ]:
def generator(noise_z):
  hidden1 = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
  hidden2 = tf.nn.relu(tf.matmul(hidden1, G_W2) + G_b2)
  hidden3 = tf.nn.relu(tf.matmul(hidden2, G_W3) + G_b3)
  output = tf.nn.sigmoid(tf.matmul(hidden3, G_W4) + G_b4)

  return output

In [ ]:
def discriminator(inputs):
  hidden1 = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
  hidden2 = tf.nn.relu(tf.matmul(hidden1, D_W2) + D_b2)
  output = tf.nn.sigmoid(tf.matmul(hidden2, D_W3) + D_b3)

  return output

In [ ]:
def get_noise(batch_size, n_noise):
  return np.random.normal(size=(batch_size, n_noise))

In [ ]:
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

In [ ]:
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene))

In [ ]:
D_var_list = [D_W1, D_b1, D_W2, D_b2, D_W3, D_b3]
G_var_list = [G_W1, G_b1, G_W2, G_b2, G_W3, G_b3, G_W4, G_b4]

In [ ]:
train_D = tf.train.RMSPropOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.RMSPropOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    noise = get_noise(batch_size, n_noise)
    
    _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Z: noise})
    _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})
  
  if epoch == 0 or (epoch + 1) % 10 == 0:
    print('Epoch:', '%04d' % epoch, 'D loss: {:.4}'.format(loss_val_D), 
        'G loss: {:.4}'.format(loss_val_G))

  # 확인용 이미지 생성

  if epoch == 0 or (epoch + 1) % 10 == 0:
    sample_size = 10
    noise = get_noise(sample_size, n_noise)
    samples = sess.run(G, feed_dict={Z: noise})

    fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

    for i in range(sample_size):
      ax[i].set_axis_off()
      ax[i].imshow(np.reshape(samples[i], (28, 28)))

    plt.savefig('./samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
    
    plt.close(fig)

print('최적화 완료!')

Epoch: 0000 D loss: -1.107 G loss: -1.514
Epoch: 0009 D loss: -1.187 G loss: -1.032
Epoch: 0019 D loss: -1.209 G loss: -1.055
Epoch: 0029 D loss: -1.115 G loss: -1.017
Epoch: 0039 D loss: -1.114 G loss: -0.9722
Epoch: 0049 D loss: -1.557 G loss: -0.5658
Epoch: 0059 D loss: -1.166 G loss: -0.7622
Epoch: 0069 D loss: -1.001 G loss: -1.426
Epoch: 0079 D loss: -0.2726 G loss: -2.814
Epoch: 0089 D loss: -0.05561 G loss: -4.536
Epoch: 0099 D loss: -0.09655 G loss: -3.646
최적화 완료!
